<a href="https://colab.research.google.com/github/springboardmentor458/OBJECT_RECOGNITION_SYSTEM_Infosys_Internship_Oct2024/blob/Harsha-Kota/springboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless albumentations torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import torch
import os
import json

Use this if you need all those images in folder

In [ ]:
"""
import json
import cv2
import urllib.request
import numpy as np
import os

json_filename = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
with open(json_filename, 'r') as f:
    data = json.load(f)

output_dir = '/content/output_images'
os.makedirs(output_dir, exist_ok=True)

def download_and_save(image_url, annotations, filename):
    resp = urllib.request.urlopen(image_url)
    img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    for ann in annotations:
        x, y, w, h = ann['bbox']
        cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), 2)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, img)

for img_data in data['images']:
    img_id = img_data['id']
    img_url = img_data['coco_url']
    filename = img_data['file_name']
    annotations = [ann for ann in data.get('annotations', []) if ann['image_id'] == img_id]
    download_and_save(img_url, annotations, filename)

print(f"All images saved in {output_dir}")
"""

Use This for Direct Representations

In [ ]:
# Cell 1
import json
import cv2
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

json_filename = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
with open(json_filename, 'r') as f:
    data = json.load(f)

def download_and_display(image_url, annotations, image=None):
    resp = urllib.request.urlopen(image_url)
    img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if image is not None:
        img = image

    for ann in annotations:
        x, y, w, h = ann['bbox']
        cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Get the user input for the number of images to load
num_images = int(input("Enter the number of images to load (default is all): ") or len(data['images']))

original_images = []
original_annotations = []

for i, img_data in enumerate(data['images']):
    if i >= num_images:
        break
    img_id = img_data['id']
    img_url = img_data['coco_url']
    annotations = [ann for ann in data.get('annotations', []) if ann['image_id'] == img_id]
    original_images.append(img_url)
    original_annotations.append(annotations)
    download_and_display(img_url, annotations)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def process_and_display_images(original_images, original_annotations):
    def draw_annotations(image, annotations):
        # Create a copy of the image to avoid modifying the original
        annotated_img = image.copy()
        for ann in annotations:
            x, y, w, h = ann['bbox']
            cv2.rectangle(annotated_img,
                         (int(x), int(y)),
                         (int(x+w), int(y+h)),
                         (255, 0, 0), 2)
        return annotated_img

    def create_image_grid(images, titles, num_cols=3):
        num_images = len(images)
        num_rows = (num_images + num_cols - 1) // num_cols

        fig = plt.figure(figsize=(15, 5 * num_rows))
        gs = GridSpec(num_rows, num_cols, figure=fig)

        for idx, (img, title) in enumerate(zip(images, titles)):
            row = idx // num_cols
            col = idx % num_cols
            ax = fig.add_subplot(gs[row, col])

            if len(img.shape) == 2:  # If grayscale
                # Convert grayscale to BGR for annotation
                img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                ax.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
            else:  # If color
                ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

            ax.set_title(title)
            ax.axis('off')

        plt.tight_layout()
        plt.show()

    for img_url, annotations in zip(original_images, original_annotations):
        # Download and process each image
        resp = urllib.request.urlopen(img_url)
        img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # Process images with different techniques
        processed_images = []
        titles = []

        # 1. Original Image with annotations
        annotated_original = draw_annotations(img, annotations)
        processed_images.append(annotated_original)
        titles.append('Original Image with Annotations')

        # 2. Binary Image with annotations
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        binary_img_color = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
        annotated_binary = draw_annotations(binary_img_color, annotations)
        processed_images.append(annotated_binary)
        titles.append('Binary Image with Annotations')

        # 3. Resized Image with annotations
        resized_img = cv2.resize(img, (224, 224))
        # Scale annotations for resized image
        scale_x = 224.0 / img.shape[1]
        scale_y = 224.0 / img.shape[0]
        scaled_annotations = []
        for ann in annotations:
            scaled_ann = ann.copy()
            x, y, w, h = ann['bbox']
            scaled_ann['bbox'] = [x * scale_x, y * scale_y, w * scale_x, h * scale_y]
            scaled_annotations.append(scaled_ann)
        annotated_resized = draw_annotations(resized_img, scaled_annotations)
        processed_images.append(annotated_resized)
        titles.append('Resized Image (224x224) with Annotations')

        # 4. Rotated Image with annotations (portrait orientation images will rotate.)
        height, width, _ = img.shape
        if height > width:              # If you want to rotate all images regardless of their orientation remove the line or adjust
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            # Adjust annotations for rotation
            rotated_annotations = []
            for ann in annotations:
                rotated_ann = ann.copy()
                x, y, w, h = ann['bbox']
                rotated_ann['bbox'] = [y, width - x - w, h, w]
                rotated_annotations.append(rotated_ann)
            rotation_status = "Rotated 90°"
            annotated_rotated = draw_annotations(rotated_img, rotated_annotations)
        else:
            rotated_img = img
            rotation_status = "No Rotation Needed"
            annotated_rotated = draw_annotations(rotated_img, annotations)
        processed_images.append(annotated_rotated)
        titles.append(f'Orientation Check\n({rotation_status})')

        # 5. Sharpened Image with annotations
        sharpened_img = cv2.GaussianBlur(rotated_img, (0, 0), sigmaX=1, sigmaY=1)
        sharpened_img = cv2.addWeighted(rotated_img, 1.5, sharpened_img, -0.5, 0)
        annotated_sharpened = draw_annotations(sharpened_img,
                                             rotated_annotations if height > width else annotations)
        processed_images.append(annotated_sharpened)
        titles.append('Sharpened Image with Annotations')

        # 6. Black and White Image with annotations
        bw_img = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2GRAY)
        bw_img_color = cv2.cvtColor(bw_img, cv2.COLOR_GRAY2BGR)
        annotated_bw = draw_annotations(bw_img_color,
                                      rotated_annotations if height > width else annotations)
        processed_images.append(annotated_bw)
        titles.append('Black & White Image with Annotations')

        # Display grid for current image
        create_image_grid(processed_images, titles)

# Usage:
processed_images, processed_annotations = process_and_display_images(original_images, original_annotations)

In [ ]:
# Cell 2(first success)
import cv2
import numpy as np
from PIL import Image

def process_images(original_images, original_annotations):
    processed_images = []
    processed_annotations = []

    for img_url, annotations in zip(original_images, original_annotations):
        resp = urllib.request.urlopen(img_url)
        img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # Filter images
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Resize images
        resized_img = cv2.resize(img, (224, 224))

        # Test for correct orientation
        height, width, _ = img.shape
        if height > width:
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        else:
            rotated_img = img

        # Fix blurry images
        sharpened_img = cv2.GaussianBlur(rotated_img, (0, 0), sigmaX=1, sigmaY=1)
        sharpened_img = cv2.addWeighted(rotated_img, 1.5, sharpened_img, -0.5, 0)

        # Apply filters (e.g., black and white)
        bw_img = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2GRAY)

        processed_images.extend([img, binary_img, resized_img, rotated_img, sharpened_img, bw_img])
        processed_annotations.extend([annotations] * 6)

    return processed_images, processed_annotations

# Call the processing function with the data from the previous cell
processed_images, processed_annotations = process_images(original_images, original_annotations)

# Display the processed images
for i, (img, annotations) in enumerate(zip(processed_images, processed_annotations)):
    download_and_display(original_images[i % len(original_images)], annotations, image=img)

In [8]:
class CocoDataset(Dataset):
    def __init__(self, images, annotations, category_mapping, img_dir, transform=None):
        self.images = images
        self.annotations = annotations
        self.category_mapping = category_mapping
        self.img_dir = img_dir
        self.transform = transform
        self.image_id_to_annotations = self._group_annotations_by_image()
    def _group_annotations_by_image(self):
        image_id_to_annotations = {}
        for ann in self.annotations:
            image_id = ann['image_id']
            if image_id not in image_id_to_annotations:
                image_id_to_annotations[image_id] = []
            image_id_to_annotations[image_id].append(ann)
        return image_id_to_annotations
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image_info = self.images[idx]
        img_path = os.path.join(self.img_dir, image_info['/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Get annotations
        image_id = image_info['id']
        annotations = self.image_id_to_annotations.get(image_id, [])

        boxes = []
        labels = []
        for ann in annotations:
            x, y, width, height = ann['bbox']
            boxes.append([x, y, x + width, y + height])
            labels.append(ann['category_id'])
      # Convert boxes and labels to numpy arrays for Albumentations
        boxes = np.array(boxes)
        labels = np.array(labels)

        # Apply transformations
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, labels=labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']

        # Convert to PyTorch tensors
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        return image, target